# 📘 Module 1: 水文循环的数字孪生 (The Digital Twin of Water Cycle)

## 学习目标 (Learning Objectives)

- 理解"降雨-径流"的基本关系
- 观察水文响应的滞后性特征
- 实现一个简单的线性水箱模型
- 理解水文模型的本质：质量守恒

---

## 🌧️ 什么是水文模型？

想象一下，当雨水落到地面上会发生什么？
- 一部分被植物截留
- 一部分渗入土壤
- 一部分形成地表径流
- 最终汇入河流...

**水文模型**就是用数学方程来描述这些复杂过程的工具。它就像是流域的"数字孪生"——一个能够模拟真实世界水循环的虚拟模型。

In [ ]:
# 首先，让我们导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

# 设置随机种子以确保结果可重复
np.random.seed(42)

print("✅ 库导入成功！让我们开始探索水文世界吧！")

## 📊 Part 1: 观察降雨-径流关系

让我们先生成一些合成数据来观察降雨和径流之间的关系。

### 思考问题 🤔
当降雨发生时，河流的水位是立刻上涨还是过一段时间才上涨？

In [ ]:
def generate_synthetic_rainfall(n_days=100):
    """
    生成合成降雨数据
    
    Parameters:
    -----------
    n_days : int
        模拟天数
        
    Returns:
    --------
    rainfall : np.ndarray
        降雨量序列 [mm/day]
    """
    # 基础：70%的天数没有降雨
    rainfall = np.zeros(n_days)
    
    # 随机选择降雨天（约30%）
    rain_days = np.random.choice(n_days, size=int(n_days * 0.3), replace=False)
    
    # 降雨量服从指数分布（大多数是小雨，少数是大雨）
    rainfall[rain_days] = np.random.exponential(scale=10, size=len(rain_days))
    
    return rainfall

# 生成100天的降雨数据
rainfall = generate_synthetic_rainfall(100)
print(f"📈 生成了 {len(rainfall)} 天的降雨数据")
print(f"   总降雨量: {rainfall.sum():.1f} mm")
print(f"   最大日降雨: {rainfall.max():.1f} mm")
print(f"   降雨天数: {np.sum(rainfall > 0)} 天")

In [ ]:
def simple_runoff_model(rainfall, k=0.2, initial_storage=50):
    """
    最简单的线性水箱模型
    
    核心公式：
    - 水量守恒：dS/dt = P - Q
    - 线性出流：Q = k × S
    
    其中：
    - S: 水箱蓄水量 [mm]
    - P: 降雨输入 [mm/day]
    - Q: 径流输出 [mm/day]
    - k: 退水系数 [1/day]
    
    Parameters:
    -----------
    rainfall : np.ndarray
        降雨序列
    k : float
        退水系数（越大响应越快）
    initial_storage : float
        初始蓄水量
        
    Returns:
    --------
    discharge : np.ndarray
        径流序列
    storage : np.ndarray
        蓄水量序列
    """
    n = len(rainfall)
    storage = np.zeros(n)
    discharge = np.zeros(n)
    
    storage[0] = initial_storage
    
    for t in range(1, n):
        # 水量输入
        storage[t] = storage[t-1] + rainfall[t]
        
        # 线性出流：Q = k × S
        discharge[t] = k * storage[t]
        
        # 更新蓄水量
        storage[t] = storage[t] - discharge[t]
    
    return discharge, storage

# 使用默认参数运行模型
discharge, storage = simple_runoff_model(rainfall, k=0.2)
print("✅ 模型运行完成！")

In [ ]:
def plot_rainfall_runoff(rainfall, discharge, title="降雨-径流双轴图"):
    """
    绘制经典的降雨-径流双轴图
    
    这种图在水文学中非常常见：
    - 上方（倒置条形图）：降雨量
    - 下方（折线图）：径流量
    """
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    days = np.arange(len(rainfall))
    
    # 创建第二个Y轴（用于降雨，倒置）
    ax2 = ax1.twinx()
    
    # 绘制降雨（倒置的条形图）
    ax2.bar(days, rainfall, color='steelblue', alpha=0.7, label='Rainfall (降雨)')
    ax2.set_ylim(ax2.get_ylim()[1] * 2.5, 0)  # 倒置Y轴
    ax2.set_ylabel('Rainfall [mm/day]', color='steelblue', fontsize=12)
    ax2.tick_params(axis='y', labelcolor='steelblue')
    
    # 绘制径流（折线图）
    ax1.plot(days, discharge, color='coral', linewidth=2, label='Discharge (径流)')
    ax1.fill_between(days, 0, discharge, color='coral', alpha=0.3)
    ax1.set_xlabel('Time [days]', fontsize=12)
    ax1.set_ylabel('Discharge [mm/day]', color='coral', fontsize=12)
    ax1.tick_params(axis='y', labelcolor='coral')
    ax1.set_xlim(0, len(rainfall))
    ax1.set_ylim(0, None)
    
    plt.title(title, fontsize=14, fontweight='bold')
    
    # 添加图例
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    
    plt.tight_layout()
    plt.show()

# 绘制降雨-径流图
plot_rainfall_runoff(rainfall, discharge, "线性水箱模型：降雨-径流响应 (k=0.2)")

### 🔍 观察要点

从上图中，你能观察到什么？

1. **滞后性 (Lag)**：降雨发生后，径流峰值是否立即出现？
2. **衰减性 (Recession)**：一场降雨后，径流是如何慢慢减小的？
3. **累积效应**：连续降雨时，径流有什么特点？

这就是水文响应的基本特征！

## 🎮 Part 2: 动手调参 - 理解参数的物理意义

现在，让我们来看看改变参数 `k` 会有什么效果。

**退水系数 k 的物理意义**：
- k 越大：水箱出水越快，径流响应越迅速
- k 越小：水箱出水越慢，径流响应越平缓

这在现实中对应什么？
- 城市流域（硬化地面多）→ k 值较大
- 森林流域（植被覆盖多）→ k 值较小

In [ ]:
# 比较不同k值的效果
k_values = [0.05, 0.1, 0.2, 0.4, 0.6]
colors = plt.cm.viridis(np.linspace(0, 0.8, len(k_values)))

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# 上图：降雨
axes[0].bar(np.arange(len(rainfall)), rainfall, color='steelblue', alpha=0.7)
axes[0].set_ylabel('Rainfall [mm/day]', fontsize=12)
axes[0].set_title('Input: Rainfall', fontsize=12)
axes[0].invert_yaxis()

# 下图：不同k值的径流响应
for k, color in zip(k_values, colors):
    discharge, _ = simple_runoff_model(rainfall, k=k)
    axes[1].plot(np.arange(len(discharge)), discharge, 
                 color=color, linewidth=2, label=f'k = {k}')

axes[1].set_xlabel('Time [days]', fontsize=12)
axes[1].set_ylabel('Discharge [mm/day]', fontsize=12)
axes[1].set_title('Output: Discharge with Different k Values', fontsize=12)
axes[1].legend(title='Recession Coefficient', loc='upper right')
axes[1].set_xlim(0, len(rainfall))

plt.suptitle('退水系数k对径流响应的影响', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 💡 思考题

1. 当k=0.6时，径流曲线有什么特点？
2. 当k=0.05时，为什么基流（baseflow）会比较高？
3. 如果你要模拟一个城市化程度很高的流域，你会选择大k还是小k？

## 📐 Part 3: 水量守恒 - 水文学的基本原理

所有水文模型的核心都是**质量守恒定律**（水量平衡方程）：

$$P - E - Q = \Delta S$$

其中：
- **P**: 降水 (Precipitation)
- **E**: 蒸发蒸腾 (Evapotranspiration)
- **Q**: 径流 (Runoff/Discharge)
- **ΔS**: 蓄水量变化 (Change in Storage)

这个公式告诉我们：**进入流域的水 = 离开流域的水 + 储存变化**

让我们验证一下我们的简单模型是否满足水量守恒！

In [ ]:
# 验证水量守恒
discharge, storage = simple_runoff_model(rainfall, k=0.2, initial_storage=50)

# 计算各项
total_rainfall = np.sum(rainfall)  # 总降雨
total_discharge = np.sum(discharge)  # 总径流
storage_change = storage[-1] - 50  # 蓄水变化（最终 - 初始）

# 水量平衡检验
balance = total_rainfall - total_discharge - storage_change

print("═" * 50)
print("💧 水量平衡检验 (Water Balance Check)")
print("═" * 50)
print(f"\n📥 输入项:")
print(f"   总降雨量 (P):        {total_rainfall:8.2f} mm")
print(f"\n📤 输出项:")
print(f"   总径流量 (Q):        {total_discharge:8.2f} mm")
print(f"\n📦 存储变化:")
print(f"   蓄水变化 (ΔS):       {storage_change:8.2f} mm")
print(f"   (初始: 50.00 mm → 最终: {storage[-1]:.2f} mm)")
print(f"\n⚖️ 平衡检验:")
print(f"   P - Q - ΔS =         {balance:8.4f} mm")
print(f"\n{'✅ 水量守恒满足！' if abs(balance) < 0.01 else '❌ 水量不守恒！'}")

In [ ]:
# 可视化蓄水量变化
fig, ax = plt.subplots(figsize=(14, 6))

days = np.arange(len(storage))

# 绘制蓄水量
ax.fill_between(days, 0, storage, color='lightblue', alpha=0.7, label='Storage (蓄水)')
ax.plot(days, storage, color='darkblue', linewidth=2)

# 标注初始和最终蓄水量
ax.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='Initial Storage')
ax.scatter([0, len(storage)-1], [50, storage[-1]], color='red', s=100, zorder=5)
ax.annotate(f'Start: 50.0 mm', (0, 50), xytext=(5, 60), fontsize=10)
ax.annotate(f'End: {storage[-1]:.1f} mm', (len(storage)-1, storage[-1]), 
            xytext=(len(storage)-15, storage[-1]+10), fontsize=10)

ax.set_xlabel('Time [days]', fontsize=12)
ax.set_ylabel('Storage [mm]', fontsize=12)
ax.set_title('水箱蓄水量随时间的变化', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.set_xlim(0, len(storage))
ax.set_ylim(0, None)

plt.tight_layout()
plt.show()

## 🧪 Part 4: 实验 - 尝试拟合"真实"数据

现在，让我们来做一个有趣的实验！

假设我们有一组"真实"的径流观测数据（实际上是用k=0.15生成的），你能通过调整k值来找到最佳拟合吗？

In [ ]:
# 生成"真实"观测数据（使用隐藏的k值）
true_k = 0.15  # 这是"真实"的k值
true_discharge, _ = simple_runoff_model(rainfall, k=true_k)

# 添加一些观测噪声
observed_discharge = true_discharge + np.random.normal(0, 0.5, len(true_discharge))
observed_discharge = np.maximum(0, observed_discharge)  # 确保非负

def calculate_nse(observed, simulated):
    """
    计算Nash-Sutcliffe效率系数
    
    NSE = 1 - Σ(Qobs - Qsim)² / Σ(Qobs - mean(Qobs))²
    
    - NSE = 1: 完美拟合
    - NSE = 0: 模型和观测均值一样好
    - NSE < 0: 模型比使用均值还差
    """
    numerator = np.sum((observed - simulated) ** 2)
    denominator = np.sum((observed - np.mean(observed)) ** 2)
    return 1 - numerator / denominator

# 尝试不同的k值
k_test_values = np.linspace(0.05, 0.4, 50)
nse_scores = []

for k in k_test_values:
    sim_discharge, _ = simple_runoff_model(rainfall, k=k)
    nse = calculate_nse(observed_discharge, sim_discharge)
    nse_scores.append(nse)

# 找到最佳k值
best_idx = np.argmax(nse_scores)
best_k = k_test_values[best_idx]
best_nse = nse_scores[best_idx]

print(f"🎯 最佳拟合结果:")
print(f"   最佳 k 值: {best_k:.4f}")
print(f"   真实 k 值: {true_k:.4f}")
print(f"   NSE得分: {best_nse:.4f}")

In [ ]:
# 可视化参数敏感性分析
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：NSE vs k
axes[0].plot(k_test_values, nse_scores, 'b-', linewidth=2)
axes[0].axvline(x=best_k, color='green', linestyle='--', label=f'Best k={best_k:.3f}')
axes[0].axvline(x=true_k, color='red', linestyle=':', label=f'True k={true_k:.3f}')
axes[0].scatter([best_k], [best_nse], color='green', s=100, zorder=5)
axes[0].set_xlabel('Recession Coefficient (k)', fontsize=12)
axes[0].set_ylabel('NSE Score', fontsize=12)
axes[0].set_title('参数敏感性分析', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 右图：观测vs模拟对比
best_sim_discharge, _ = simple_runoff_model(rainfall, k=best_k)
days = np.arange(len(observed_discharge))

axes[1].plot(days, observed_discharge, 'ko', markersize=3, alpha=0.5, label='Observed')
axes[1].plot(days, best_sim_discharge, 'b-', linewidth=2, label=f'Simulated (k={best_k:.3f})')
axes[1].set_xlabel('Time [days]', fontsize=12)
axes[1].set_ylabel('Discharge [mm/day]', fontsize=12)
axes[1].set_title(f'观测与模拟对比 (NSE={best_nse:.3f})', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('模型校准结果', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 📚 Part 5: 总结与延伸

### 我们学到了什么？

1. **水文模型的本质**：用数学方程描述水循环过程
2. **质量守恒**：所有水文模型的基础 (P - E - Q = ΔS)
3. **线性水箱模型**：最简单的概念模型 (Q = k × S)
4. **参数校准**：通过调整参数使模型拟合观测数据

### 延伸思考 🤔

1. 真实流域比我们的简单模型复杂得多，还需要考虑哪些过程？
   - 蒸发蒸腾
   - 雪的累积和融化
   - 土壤水分动态
   - 地下水补给

2. 为什么一个参数的模型可能不够用？
   - 不同水文过程需要不同的参数来描述
   - 时间尺度不同（快速径流 vs 基流）

### 下一步学习 📖

在下一个模块中，我们将探索更复杂的物理模型（如HBV模型），
看看如何用多个"水箱"来描述不同的水文过程！

---

## 🎯 课后练习

1. **修改代码**：尝试在模型中加入蒸发过程（假设每天蒸发2mm），观察对水量平衡的影响

2. **思考问题**：
   - 为什么我们找到的最佳k值和真实k值略有不同？
   - 如果观测数据的噪声更大，校准结果会怎样变化？

3. **探索实验**：
   - 使用更长的时间序列（如1000天）重复校准实验
   - 比较短期数据和长期数据的校准效果

In [ ]:
# 🎓 练习区域 - 在这里编写你的代码

# 练习1: 加入蒸发过程的模型
def runoff_model_with_evaporation(rainfall, k=0.2, evap=2.0, initial_storage=50):
    """
    TODO: 修改simple_runoff_model，加入蒸发过程
    
    提示：在每个时间步，需要从蓄水量中减去蒸发量
    注意：蒸发量不能超过当前蓄水量
    """
    # 在这里编写你的代码
    pass

# 练习2: 测试你的代码
# discharge_with_evap, storage_with_evap = runoff_model_with_evaporation(rainfall)
# 绘制对比图...